# core

> API to get structured output from a string using different LLM providers

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
from litellm import completion
from pydantic import BaseModel

In [ ]:
#| export
def structured_output(model:str, # Model name, see examples here or LiteLLM docs for complete list
                      system_prompt:str, # Instructions for LLM to process the input string
                      user_prompt:str, # Input string that will be processed
                      response_format:BaseModel # User-defined Pydantic model to define output 
                     ) -> BaseModel :
    response = completion(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        response_format=response_format
    )
    return response_format.model_validate_json(response.choices[0].message.content)

In [ ]:
model="azure/gpt-4o-2024-08-06"
system_prompt = "Extract the event information."

In [ ]:
# This defines the output response format that we want
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [ ]:
user_prompt = "Alice and Bob are going to Carmen's Birtday party on 22nd March 2025"
# Note that the word "Birthday" is spelt incorrectly on purpose for this example

In [ ]:
#| eval: false
r = structured_output(model=model,
                      system_prompt=system_prompt,
                      user_prompt=user_prompt,
                      response_format=CalendarEvent, #Note this is the class name (without the `()`)
                 )

r.model_dump()

{'name': "Carmen's Birthday Party",
 'date': '22nd March 2025',
 'participants': ['Alice', 'Bob']}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()